In [2]:
from tqdm.auto import tqdm 
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nf
from netCDF4 import Dataset
%matplotlib inline
import glob
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [3]:
datapath="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/Data/ECMWF/ERA5_25kmx3hr/"
path="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/"
output="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/outputs/"
target="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/outputs/targets/"

In [122]:
tracklist = sorted(glob.glob(path+'tracks_wp/*_2011*'))

In [123]:
tracklist

['/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20110519.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20110616.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20110926.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20111211.csv']

In [119]:
dm1=xr.open_mfdataset([datapath+'/vor/vor_2011.nc',datapath+'/rhum/rhum_2011.nc'])

In [120]:
#ssingle level variables
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def largearea(dataset=None,invar=None,indices=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    var_out=np.zeros((len(indices),64,64))
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_orad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:64,0:64]
    return var_out

def largearea_withpres(dataset=None,invar=None,indices=None):
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","plev","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices))),
               plev=(["plev"],dataset.plev.data)),
    attrs=dict(description="coords with matrices"),)
    var_out=np.zeros((len(indices),len(dm1.plev.data),64,64))
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_orad[it,:]
        for ip in range(len(dm1.plev.data)):
            try:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
            except:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:64,0:64]
    return var_out 

In [124]:
###
def createmask(dm=None,irad=None,orad=None,lonselect=None,latselect=None):
    mask = []
    for ti in range(len(tc_irad)):
        lonselect = dm.lon.sel(lon=slice(orad[ti,:][2],orad[ti,:][3])).data
        latselect = np.flipud(dm.lat.sel(lat=slice(orad[ti,:][1],orad[ti,:][0])).data)
        if (lonselect.shape != 64) or (latselect.shape != 64):
            lon2d,lat2d = np.meshgrid(lonselect[0:64],latselect[0:64])
        else:
            lon2d,lat2d = np.meshgrid(lonselect,latselect)
        #############################################################################################
        latcriteria = np.logical_and(lat2d>irad[ti][0],lat2d<irad[ti][1])
        loncriteria = np.logical_and(lon2d>irad[ti][2],lon2d<irad[ti][3])
        allcriteria = np.logical_and(loncriteria,latcriteria)
        mask.append(allcriteria)
    return mask

def readyear(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/svars_'+str(year)+'.nc')
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2011)

TCs_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallsvarout = [largearea(dm2,mysvar[i],indices_store) for i in (range(len(mysvar)))]
    svarname1 = ['outu10','outv10','out2mdewtmp','out2mtmp','outconv_ppt','outtot_cld_ice',\
           'outtot_cldwtr','outtot_cld_rain','outvi_div_cld_froz_wtr','outvi_div_cld_liq_wtr','outvi_div_gpot_flux',\
           'outvi_div_ke_flux','outvi_div_mass_flux','outvi_div_moisture_flux','outvi_div_olr_flux','outvi_div_tot_enrgy_flux',\
           'outvi_ke','outvi_pe_inte','outvi_pe_ie_latentenrgy','outvi_temp','outvi_olr','outvi_tot_enrgy','outvi_moisture_div']
   
    svarname = ['u10','v10','2mdewtmp','2mtmp','conv_ppt','tot_cld_ice',\
           'tot_cldwtr','tot_cld_rain','vi_div_cld_froz_wtr','vi_div_cld_liq_wtr','vi_div_gpot_flux',\
           'vi_div_ke_flux','vi_div_mass_flux','vi_div_moisture_flux','vi_div_olr_flux','vi_div_tot_enrgy_flux',\
           'vi_ke','vi_pe_inte','vi_pe_ie_latentenrgy','vi_temp','vi_olr','vi_tot_enrgy','vi_moisture_div']
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    lonselect = dm2.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)

    mymask = createmask(dm=dm2,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...][~mymask[i]] for i in range(len(mymask))]
        tsdict[svarname1[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCs_ts.append(tsdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/4 [00:00<?, ?it/s]

In [125]:
###radiation variables
def createmask(dm=None,irad=None,orad=None,lonselect=None,latselect=None):
    mask = []
    for ti in range(len(tc_irad)):
        lonselect = dm.lon.sel(lon=slice(orad[ti,:][2],orad[ti,:][3])).data
        latselect = np.flipud(dm.lat.sel(lat=slice(orad[ti,:][1],orad[ti,:][0])).data)
        if (lonselect.shape != 64) or (latselect.shape != 64):
            lon2d,lat2d = np.meshgrid(lonselect[0:64],latselect[0:64])
        else:
            lon2d,lat2d = np.meshgrid(lonselect,latselect)
        #############################################################################################
        latcriteria = np.logical_and(lat2d>irad[ti][0],lat2d<irad[ti][1])
        loncriteria = np.logical_and(lon2d>irad[ti][2],lon2d<irad[ti][3])
        allcriteria = np.logical_and(loncriteria,latcriteria)
        mask.append(allcriteria)
    return mask

def readyear(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/radvars_'+str(year)+'.nc')
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2011)

TCr_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallsvarout = [largearea(dm2,mysvar[i],indices_store) for i in (range(len(mysvar)))]
    
    svarname1 = ['outcape','outinst_10m_wnd_gst','outinst_moisture_flux','outinst_ssh_flux','outsurfmean_swr_flux','outsurfmean_lhf',\
           'outsurfmean_lwr_flux','outsurfmean_shf','outdwnwrdmean_swr_flux','outtopmean_lwr_flux','outtopmean_swr_flux',\
           'outvimean_moisture_div','outsurf_lhf','outsurf_shf','outtot_suprcool_liqwtr','outtot_wtr_vpr']
    
    
    svarname = ['cape','inst_10m_wnd_gst','inst_moisture_flux','inst_ssh_flux','surfmean_swr_flux','surfmean_lhf',\
           'surfmean_lwr_flux','surfmean_shf','dwnwrdmean_swr_flux','topmean_lwr_flux','topmean_swr_flux',\
           'vimean_moisture_div','surf_lhf','surf_shf','tot_suprcool_liqwtr','tot_wtr_vpr']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    lonselect = dm2.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)

    mymask = createmask(dm=dm2,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...][~mymask[i]] for i in range(len(mymask))]
        tsdict[svarname1[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCr_ts.append(tsdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/4 [00:00<?, ?it/s]

In [126]:
#########precip, mslp, sst variables############
def createmask(dm=None,irad=None,orad=None,lonselect=None,latselect=None):
    mask = []
    for ti in range(len(tc_irad)):
        lonselect = dm.lon.sel(lon=slice(orad[ti,:][2],orad[ti,:][3])).data
        latselect = np.flipud(dm.lat.sel(lat=slice(orad[ti,:][1],orad[ti,:][0])).data)
        if (lonselect.shape != 64) or (latselect.shape != 64):
            lon2d,lat2d = np.meshgrid(lonselect[0:64],latselect[0:64])
        else:
            lon2d,lat2d = np.meshgrid(lonselect,latselect)
        #############################################################################################
        latcriteria = np.logical_and(lat2d>irad[ti][0],lat2d<irad[ti][1])
        loncriteria = np.logical_and(lon2d>irad[ti][2],lon2d<irad[ti][3])
        allcriteria = np.logical_and(loncriteria,latcriteria)
        mask.append(allcriteria)
    return mask

def readyear(year=None):
    dm2 = xr.open_mfdataset([datapath+'/prate/prates_2011.nc',datapath+'/mslp/mslp_2011.nc',datapath+'/sst/sst_2011.nc'])
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2011)

TCpr_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallsvarout = [largearea(dm2,mysvar[i],indices_store) for i in (range(len(mysvar)))]
    
    svarname1 = ['outmslp','outconv_rrate','outls_rrate','outmn_conv_prate',\
                 'outmn_ls_prate','outmn_tot_prate','outsst']
    
    svarname = ['mslp','conv_rrate','ls_rrate','mn_conv_prate','mn_ls_prate','mn_tot_prate','sst']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    lonselect = dm2.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)

    mymask = createmask(dm=dm2,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...][~mymask[i]] for i in range(len(mymask))]
        tsdict[svarname1[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCpr_ts.append(tsdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/4 [00:00<?, ?it/s]

In [127]:
#Pressure vars - vor hum
TCp_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outvort','outrhum']
    pvarname = ['vort','rhum']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm1.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm1.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm1,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCp_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/4 [00:00<?, ?it/s]

In [128]:
dm1=xr.open_dataset(datapath+'/gpot/gpot_2011.nc')

In [129]:
# Pressure vars - geopotential
TCgp_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outgpot']
    pvarname = ['gpot']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm1.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm1.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm1,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCgp_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/4 [00:00<?, ?it/s]

In [130]:
dm1=xr.open_dataset(datapath+'/vvel/w_wnd_2011.nc')

In [131]:
#Pressure vars - vertical velocity
TCvv_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outvvel']
    pvarname = ['vvel']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm1.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm1.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm1,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCvv_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/4 [00:00<?, ?it/s]

In [132]:
preslv = [str(int(obj)) for obj in dm1.plev.data/100]

In [133]:
dx=xr.open_dataset(datapath+'/tlevs/tlevs_2011.nc')

In [134]:
plev=[str(int(obj)) for obj in dx.plev.data/100]

In [135]:
tracklist

['/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20110519.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20110616.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20110926.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20111211.csv']

In [136]:
#storm 1 songda

maindf2=pd.read_csv(output+'inner/inner_wp_all_songda.csv',delimiter=r",")
maindf2.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf2=maindf2.drop('a', axis=1)

pdf_songda=pd.read_csv(target+'2011_pmin_nwp_songda.csv',delimiter=r",")
pdf_songda.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_songda=pdf_songda.drop('a', axis=1)

extra=pd.read_csv(output+'nwp/outder_2011_nwp_songda.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_songda=pd.DataFrame.from_dict(TCr_ts[0])
prlev_songda=pd.DataFrame.from_dict(TCpr_ts[0])
slev_songda=pd.DataFrame.from_dict(TCs_ts[0])

prlev_songda['outmslp']=prlev_songda['outmslp']/100

ts1_songda=pd.concat([maindf2,extra,slev_songda,rlev_songda,prlev_songda], axis=1, join='inner')


tempvortdict = {'outvort_'+plev[i]:TCp_ts[0]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[0]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[0]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[0]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}
plev_songda=pd.DataFrame.from_dict(alldict)
ts_songda=pd.concat([pdf_songda,ts1_songda,plev_songda], axis=1, join='inner')
ts_songda

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1006.396875,10.718504,0.000077,0.000052,2.137697e-05,0.000005,-0.000017,-0.000020,336.092651,350.741821,...,-0.004908,-0.002240,-0.001853,0.001826,0.008132,0.011321,0.005151,-0.001681,-0.003106,-0.002132
1,1006.892500,11.079134,0.000071,0.000047,1.221514e-05,-0.000009,-0.000033,-0.000018,336.750488,350.423828,...,0.007259,0.007266,-0.012067,-0.015655,-0.004133,0.001479,0.001095,0.002173,0.006753,0.010418
2,1006.718750,11.393514,0.000065,0.000046,3.128390e-06,-0.000018,-0.000024,-0.000004,335.092499,350.678101,...,0.009000,0.005934,-0.018071,-0.021488,-0.012283,-0.015813,-0.025527,-0.019184,-0.010112,-0.004971
3,1005.838750,10.368273,0.000058,0.000034,2.973929e-06,-0.000022,-0.000009,0.000010,335.330933,351.062988,...,0.005376,-0.002117,-0.020920,-0.015535,-0.012826,-0.024543,-0.038835,-0.031240,-0.021461,-0.016003
4,1004.691250,9.976253,0.000058,0.000047,2.414852e-06,-0.000023,-0.000002,0.000009,336.175537,351.966309,...,0.008413,0.000196,-0.014053,-0.009469,-0.023090,-0.040590,-0.049847,-0.034459,-0.018748,-0.009929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,971.403125,31.007729,0.000043,0.000016,1.074035e-05,0.000011,0.000015,0.000020,327.064941,357.946259,...,-0.065419,-0.081751,-0.128258,-0.120540,-0.101559,-0.068142,-0.020486,-0.016354,-0.013914,-0.009089
72,973.220625,27.007444,0.000033,0.000009,1.288182e-05,0.000018,0.000014,0.000017,326.311798,357.456329,...,-0.057712,-0.079964,-0.123770,-0.113536,-0.086659,-0.049811,-0.026824,-0.020744,-0.012563,-0.004011
73,978.041250,26.172678,0.000025,0.000010,-7.005151e-07,0.000016,0.000006,0.000018,322.962097,356.528564,...,-0.083257,-0.112429,-0.160784,-0.157399,-0.117797,-0.059528,-0.028926,-0.012659,-0.002795,0.008964
74,985.281875,24.936945,-0.000014,-0.000003,-2.516269e-06,0.000002,0.000021,0.000014,318.323547,356.221191,...,-0.101342,-0.135463,-0.192159,-0.187438,-0.133614,-0.079278,-0.039721,-0.020780,-0.011151,-0.002791


In [137]:
df1=ts_songda
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)
df1

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1006.396875,10.718504,0.000077,0.000052,2.137697e-05,0.000005,-0.000017,-0.000020,336.092651,350.741821,...,-0.004908,-0.002240,-0.001853,0.001826,0.008132,0.011321,0.005151,-0.001681,-0.003106,-0.002132
1,1006.892500,11.079134,0.000071,0.000047,1.221514e-05,-0.000009,-0.000033,-0.000018,336.750488,350.423828,...,0.007259,0.007266,-0.012067,-0.015655,-0.004133,0.001479,0.001095,0.002173,0.006753,0.010418
2,1006.718750,11.393514,0.000065,0.000046,3.128390e-06,-0.000018,-0.000024,-0.000004,335.092499,350.678101,...,0.009000,0.005934,-0.018071,-0.021488,-0.012283,-0.015813,-0.025527,-0.019184,-0.010112,-0.004971
3,1005.838750,10.368273,0.000058,0.000034,2.973929e-06,-0.000022,-0.000009,0.000010,335.330933,351.062988,...,0.005376,-0.002117,-0.020920,-0.015535,-0.012826,-0.024543,-0.038835,-0.031240,-0.021461,-0.016003
4,1004.691250,9.976253,0.000058,0.000047,2.414852e-06,-0.000023,-0.000002,0.000009,336.175537,351.966309,...,0.008413,0.000196,-0.014053,-0.009469,-0.023090,-0.040590,-0.049847,-0.034459,-0.018748,-0.009929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,971.403125,31.007729,0.000043,0.000016,1.074035e-05,0.000011,0.000015,0.000020,327.064941,357.946259,...,-0.065419,-0.081751,-0.128258,-0.120540,-0.101559,-0.068142,-0.020486,-0.016354,-0.013914,-0.009089
72,973.220625,27.007444,0.000033,0.000009,1.288182e-05,0.000018,0.000014,0.000017,326.311798,357.456329,...,-0.057712,-0.079964,-0.123770,-0.113536,-0.086659,-0.049811,-0.026824,-0.020744,-0.012563,-0.004011
73,978.041250,26.172678,0.000025,0.000010,-7.005151e-07,0.000016,0.000006,0.000018,322.962097,356.528564,...,-0.083257,-0.112429,-0.160784,-0.157399,-0.117797,-0.059528,-0.028926,-0.012659,-0.002795,0.008964
74,985.281875,24.936945,-0.000014,-0.000003,-2.516269e-06,0.000002,0.000021,0.000014,318.323547,356.221191,...,-0.101342,-0.135463,-0.192159,-0.187438,-0.133614,-0.079278,-0.039721,-0.020780,-0.011151,-0.002791


In [138]:
df1.to_csv(output+'timeseries_all_wp_songda.csv')

In [139]:
maindf2=pd.read_csv(output+'inner/inner_wp_all_2011_haima.csv',delimiter=r",")
maindf2.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf2=maindf2.drop('a', axis=1)

pdf_haima=pd.read_csv(target+'2011_pmin_nwp_haima.csv',delimiter=r",")
pdf_haima.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_haima=pdf_haima.drop('a', axis=1)

extra=pd.read_csv(output+'nwp/outder_2011_nwp_haima.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_haima=pd.DataFrame.from_dict(TCr_ts[1])
prlev_haima=pd.DataFrame.from_dict(TCpr_ts[1])
slev_haima=pd.DataFrame.from_dict(TCs_ts[1])

prlev_haima['outmslp']=prlev_haima['outmslp']/100

ts1_haima=pd.concat([maindf2,extra,slev_haima,rlev_haima,prlev_haima], axis=1, join='inner')


tempvortdict = {'outvort_'+plev[i]:TCp_ts[1]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[1]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[1]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[1]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}
plev_haima=pd.DataFrame.from_dict(alldict)
ts_haima=pd.concat([pdf_haima,ts1_haima,plev_haima], axis=1, join='inner')
ts_haima

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1005.852500,12.920896,0.000031,0.000026,0.000012,0.000005,0.000005,-1.391179e-06,332.352936,350.438232,...,-0.079002,-0.137651,-0.194047,-0.194255,-0.164649,-0.141096,-0.086648,-0.047737,-0.029589,-0.026193
1,1005.700625,11.155981,0.000024,0.000034,0.000013,0.000004,0.000003,-8.728022e-06,334.635468,350.661316,...,-0.084704,-0.131361,-0.176636,-0.171785,-0.134019,-0.100633,-0.042598,-0.006577,0.007843,0.010644
2,1007.673125,11.616890,0.000027,0.000020,0.000017,0.000007,-0.000006,-1.252973e-06,335.600983,350.385376,...,-0.097242,-0.126124,-0.139261,-0.125529,-0.082187,-0.053472,-0.016741,0.002478,0.009895,0.012391
3,1007.952500,10.947051,0.000032,0.000002,0.000011,0.000005,-0.000007,2.637102e-06,336.121765,350.589172,...,-0.079847,-0.111722,-0.133617,-0.133093,-0.099037,-0.067667,-0.038988,-0.021792,-0.013257,-0.010861
4,1005.093125,12.182612,0.000046,-0.000007,-0.000002,0.000001,-0.000003,-2.026347e-07,335.806244,350.569824,...,-0.054400,-0.088710,-0.120767,-0.129176,-0.105655,-0.073333,-0.045038,-0.028012,-0.018689,-0.015749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,995.521875,11.428171,0.000028,0.000025,0.000015,0.000011,0.000008,-2.628424e-07,343.384369,358.291931,...,-0.085930,-0.092569,-0.063188,-0.060228,-0.070938,-0.067992,-0.040063,-0.036365,-0.032305,-0.028503
73,995.124375,11.366698,0.000031,0.000024,0.000018,0.000009,0.000004,-1.728417e-06,341.531067,358.682953,...,-0.102788,-0.097935,-0.067236,-0.060569,-0.069522,-0.068034,-0.046914,-0.033334,-0.023815,-0.017262
74,997.292500,9.420424,0.000034,0.000030,0.000008,0.000005,0.000014,4.283335e-06,341.135956,357.978088,...,-0.084702,-0.081494,-0.048730,-0.032897,-0.037663,-0.043055,-0.024676,-0.006163,0.006478,0.012930
75,999.440000,9.506091,0.000036,0.000033,0.000014,0.000017,0.000011,-1.208833e-06,339.284729,357.697754,...,-0.074973,-0.071007,-0.029270,-0.019515,-0.032206,-0.035976,-0.024384,-0.007490,0.008387,0.017482


In [140]:
df1=ts_haima
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)
df1

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1005.852500,12.920896,0.000031,0.000026,0.000012,0.000005,0.000005,-1.391179e-06,332.352936,350.438232,...,-0.079002,-0.137651,-0.194047,-0.194255,-0.164649,-0.141096,-0.086648,-0.047737,-0.029589,-0.026193
1,1005.700625,11.155981,0.000024,0.000034,0.000013,0.000004,0.000003,-8.728022e-06,334.635468,350.661316,...,-0.084704,-0.131361,-0.176636,-0.171785,-0.134019,-0.100633,-0.042598,-0.006577,0.007843,0.010644
2,1007.673125,11.616890,0.000027,0.000020,0.000017,0.000007,-0.000006,-1.252973e-06,335.600983,350.385376,...,-0.097242,-0.126124,-0.139261,-0.125529,-0.082187,-0.053472,-0.016741,0.002478,0.009895,0.012391
3,1007.952500,10.947051,0.000032,0.000002,0.000011,0.000005,-0.000007,2.637102e-06,336.121765,350.589172,...,-0.079847,-0.111722,-0.133617,-0.133093,-0.099037,-0.067667,-0.038988,-0.021792,-0.013257,-0.010861
4,1005.093125,12.182612,0.000046,-0.000007,-0.000002,0.000001,-0.000003,-2.026347e-07,335.806244,350.569824,...,-0.054400,-0.088710,-0.120767,-0.129176,-0.105655,-0.073333,-0.045038,-0.028012,-0.018689,-0.015749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,995.521875,11.428171,0.000028,0.000025,0.000015,0.000011,0.000008,-2.628424e-07,343.384369,358.291931,...,-0.085930,-0.092569,-0.063188,-0.060228,-0.070938,-0.067992,-0.040063,-0.036365,-0.032305,-0.028503
73,995.124375,11.366698,0.000031,0.000024,0.000018,0.000009,0.000004,-1.728417e-06,341.531067,358.682953,...,-0.102788,-0.097935,-0.067236,-0.060569,-0.069522,-0.068034,-0.046914,-0.033334,-0.023815,-0.017262
74,997.292500,9.420424,0.000034,0.000030,0.000008,0.000005,0.000014,4.283335e-06,341.135956,357.978088,...,-0.084702,-0.081494,-0.048730,-0.032897,-0.037663,-0.043055,-0.024676,-0.006163,0.006478,0.012930
75,999.440000,9.506091,0.000036,0.000033,0.000014,0.000017,0.000011,-1.208833e-06,339.284729,357.697754,...,-0.074973,-0.071007,-0.029270,-0.019515,-0.032206,-0.035976,-0.024384,-0.007490,0.008387,0.017482


In [141]:
df1.to_csv(output+'timeseries_all_wp_2011_haima.csv')

In [142]:
maindf2=pd.read_csv(output+'inner/inner_wp_all_nalgae.csv',delimiter=r",")
maindf2.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf2=maindf2.drop('a', axis=1)

pdf_nalgae=pd.read_csv(target+'2011_pmin_nwp_nalgae.csv',delimiter=r",")
pdf_nalgae.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_nalgae=pdf_nalgae.drop('a', axis=1)

extra=pd.read_csv(output+'nwp/outder_2011_nwp_nalgae.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_nalgae=pd.DataFrame.from_dict(TCr_ts[2])
prlev_nalgae=pd.DataFrame.from_dict(TCpr_ts[2])
slev_nalgae=pd.DataFrame.from_dict(TCs_ts[2])

prlev_nalgae['outmslp']=prlev_nalgae['outmslp']/100

ts1_nalgae=pd.concat([maindf2,extra,slev_nalgae,rlev_nalgae,prlev_nalgae], axis=1, join='inner')


tempvortdict = {'outvort_'+plev[i]:TCp_ts[2]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[2]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[2]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[2]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}
plev_nalgae=pd.DataFrame.from_dict(alldict)
ts_nalgae=pd.concat([pdf_nalgae,ts1_nalgae,plev_nalgae], axis=1, join='inner')
ts_nalgae

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1007.563750,14.322814,6.940629e-05,3.870234e-05,-4.192808e-06,-2.194361e-05,-1.767585e-05,-1.008410e-05,335.620361,351.562347,...,0.002117,-0.001891,-0.017479,-0.025497,-0.034691,-0.031973,-0.025134,-0.014964,-0.007665,-0.003908
1,1006.638750,12.952862,5.659458e-05,3.064299e-05,-8.663010e-07,-5.622924e-06,-1.080651e-05,-8.188302e-06,336.041443,351.699280,...,0.007664,0.000347,-0.012914,-0.022190,-0.036336,-0.037121,-0.027904,-0.019399,-0.014949,-0.012632
2,1004.270000,11.460491,4.792262e-05,2.876717e-05,6.633320e-06,4.266361e-06,-6.333008e-06,-1.045010e-05,336.822693,351.721069,...,-0.009968,-0.023955,-0.032755,-0.037829,-0.044448,-0.037372,-0.013653,-0.006322,-0.005697,-0.005819
3,1004.523125,11.687129,5.025869e-05,3.547810e-05,1.158262e-05,5.285798e-06,-7.670015e-06,-1.092796e-05,336.804626,351.301819,...,-0.010994,-0.029551,-0.042272,-0.048829,-0.055289,-0.046235,-0.009643,0.005671,0.011353,0.013384
4,1005.933750,14.077259,7.611053e-05,3.487488e-05,-6.048032e-06,-4.199692e-07,1.948070e-06,-9.135704e-06,336.892578,352.463776,...,-0.044584,-0.060119,-0.053393,-0.051328,-0.049021,-0.038874,-0.013664,-0.002699,0.000349,0.000894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,1004.456875,15.985020,-4.544082e-06,-6.333099e-06,5.510799e-06,2.582844e-06,5.426166e-06,-4.337699e-08,311.929688,350.406128,...,-0.003449,-0.018637,-0.039426,-0.061876,-0.090087,-0.090623,-0.038307,0.001414,0.021520,0.029962
69,1005.758750,14.828885,6.460446e-06,-3.414246e-06,1.175802e-05,-4.896817e-06,1.916140e-06,1.696170e-06,312.387817,350.122833,...,-0.012946,-0.023763,-0.045713,-0.074294,-0.103984,-0.103562,-0.062905,-0.029687,-0.014364,-0.008179
70,1004.231250,13.743028,2.002344e-06,5.525781e-07,1.222747e-05,-5.748053e-06,2.975128e-06,9.103449e-07,312.428162,350.013916,...,-0.034433,-0.042414,-0.054894,-0.073255,-0.093079,-0.090938,-0.059798,-0.033781,-0.025972,-0.023195
71,1003.859375,13.305566,-5.314167e-07,5.844769e-06,8.193862e-06,-4.943665e-06,-2.566430e-07,1.341856e-06,309.987518,349.912048,...,-0.030328,-0.035163,-0.041638,-0.055643,-0.069267,-0.063796,-0.033468,-0.006621,0.002727,0.006328


In [143]:
df1=ts_nalgae
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)
df1

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1007.563750,14.322814,6.940629e-05,3.870234e-05,-4.192808e-06,-2.194361e-05,-1.767585e-05,-1.008410e-05,335.620361,351.562347,...,0.002117,-0.001891,-0.017479,-0.025497,-0.034691,-0.031973,-0.025134,-0.014964,-0.007665,-0.003908
1,1006.638750,12.952862,5.659458e-05,3.064299e-05,-8.663010e-07,-5.622924e-06,-1.080651e-05,-8.188302e-06,336.041443,351.699280,...,0.007664,0.000347,-0.012914,-0.022190,-0.036336,-0.037121,-0.027904,-0.019399,-0.014949,-0.012632
2,1004.270000,11.460491,4.792262e-05,2.876717e-05,6.633320e-06,4.266361e-06,-6.333008e-06,-1.045010e-05,336.822693,351.721069,...,-0.009968,-0.023955,-0.032755,-0.037829,-0.044448,-0.037372,-0.013653,-0.006322,-0.005697,-0.005819
3,1004.523125,11.687129,5.025869e-05,3.547810e-05,1.158262e-05,5.285798e-06,-7.670015e-06,-1.092796e-05,336.804626,351.301819,...,-0.010994,-0.029551,-0.042272,-0.048829,-0.055289,-0.046235,-0.009643,0.005671,0.011353,0.013384
4,1005.933750,14.077259,7.611053e-05,3.487488e-05,-6.048032e-06,-4.199692e-07,1.948070e-06,-9.135704e-06,336.892578,352.463776,...,-0.044584,-0.060119,-0.053393,-0.051328,-0.049021,-0.038874,-0.013664,-0.002699,0.000349,0.000894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,1004.456875,15.985020,-4.544082e-06,-6.333099e-06,5.510799e-06,2.582844e-06,5.426166e-06,-4.337699e-08,311.929688,350.406128,...,-0.003449,-0.018637,-0.039426,-0.061876,-0.090087,-0.090623,-0.038307,0.001414,0.021520,0.029962
69,1005.758750,14.828885,6.460446e-06,-3.414246e-06,1.175802e-05,-4.896817e-06,1.916140e-06,1.696170e-06,312.387817,350.122833,...,-0.012946,-0.023763,-0.045713,-0.074294,-0.103984,-0.103562,-0.062905,-0.029687,-0.014364,-0.008179
70,1004.231250,13.743028,2.002344e-06,5.525781e-07,1.222747e-05,-5.748053e-06,2.975128e-06,9.103449e-07,312.428162,350.013916,...,-0.034433,-0.042414,-0.054894,-0.073255,-0.093079,-0.090938,-0.059798,-0.033781,-0.025972,-0.023195
71,1003.859375,13.305566,-5.314167e-07,5.844769e-06,8.193862e-06,-4.943665e-06,-2.566430e-07,1.341856e-06,309.987518,349.912048,...,-0.030328,-0.035163,-0.041638,-0.055643,-0.069267,-0.063796,-0.033468,-0.006621,0.002727,0.006328


In [144]:
df1.to_csv(output+'timeseries_all_wp_2011_nalgae.csv')

In [145]:
maindf2=pd.read_csv(output+'inner/inner_wp_all_washi.csv',delimiter=r",")
maindf2.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf2=maindf2.drop('a', axis=1)

pdf_washi=pd.read_csv(target+'2011_pmin_nwp_washi.csv',delimiter=r",")
pdf_washi.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_washi=pdf_washi.drop('a', axis=1)

extra=pd.read_csv(output+'nwp/outder_2011_nwp_washi.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_washi=pd.DataFrame.from_dict(TCr_ts[3])
prlev_washi=pd.DataFrame.from_dict(TCpr_ts[3])
slev_washi=pd.DataFrame.from_dict(TCs_ts[3])

prlev_washi['outmslp']=prlev_washi['outmslp']/100

ts1_washi=pd.concat([maindf2,extra,slev_washi,rlev_washi,prlev_washi], axis=1, join='inner')


tempvortdict = {'outvort_'+plev[i]:TCp_ts[3]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[3]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[3]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[3]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}
plev_washi=pd.DataFrame.from_dict(alldict)
ts_washi=pd.concat([pdf_washi,ts1_washi,plev_washi], axis=1, join='inner')
ts_washi

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1007.069375,11.374318,-0.000024,6.228312e-06,2.071853e-05,0.000008,-9.523897e-06,-2.231343e-07,314.772888,350.313934,...,-0.008641,-0.022980,-0.019074,-0.026559,-0.027983,-0.029231,-0.018274,-0.007241,0.000964,0.005387
1,1008.814375,12.570400,-0.000014,2.781810e-06,1.150798e-05,0.000003,-8.442781e-06,6.725560e-06,313.111267,350.093811,...,0.003057,-0.015082,-0.013882,-0.015330,-0.021173,-0.027360,-0.008871,0.003124,0.012757,0.017991
2,1009.147500,12.993572,-0.000005,-2.801417e-07,-2.842421e-06,-0.000002,-3.891363e-06,8.958064e-06,313.254547,350.107147,...,0.006204,-0.012347,-0.032648,-0.045578,-0.049795,-0.049760,-0.033698,-0.027660,-0.020196,-0.015848
3,1007.325625,12.251040,-0.000002,1.103165e-05,-8.869802e-08,0.000004,1.414512e-06,6.908057e-06,317.138824,349.477478,...,0.018165,0.000719,-0.023445,-0.042156,-0.051826,-0.055572,-0.049214,-0.041706,-0.030513,-0.024079
4,1006.491875,12.024080,-0.000011,9.125998e-06,3.205580e-06,0.000008,7.640495e-07,6.252454e-06,317.027039,349.305023,...,0.023761,0.008616,-0.004516,-0.013878,-0.015859,-0.014243,-0.010377,-0.004005,0.004443,0.009003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,1004.050000,18.803926,0.000039,1.170732e-05,-2.679585e-07,-0.000002,-7.172116e-07,-4.642721e-06,335.051941,353.788757,...,-0.053408,-0.039242,-0.028068,-0.032141,-0.050734,-0.064892,-0.056993,-0.041778,-0.025674,-0.016432
57,1003.718750,18.780231,0.000031,-3.424115e-06,-1.704585e-05,-0.000010,7.581883e-06,-1.079770e-06,334.113342,353.285675,...,-0.052267,-0.036092,-0.027530,-0.039496,-0.054264,-0.052381,-0.032866,-0.013568,0.003633,0.013292
58,1006.248125,18.004267,0.000015,-5.888951e-06,-1.915601e-05,-0.000010,7.138413e-06,6.689090e-07,331.277893,352.275787,...,-0.046109,-0.043566,-0.049935,-0.058294,-0.056933,-0.039258,-0.023833,-0.007506,0.008584,0.017972
59,1006.571250,16.774679,0.000011,-1.164030e-05,-1.414149e-05,-0.000006,2.098375e-06,-2.144409e-06,334.114502,352.113495,...,-0.057529,-0.053221,-0.055181,-0.069872,-0.074319,-0.065659,-0.053174,-0.038931,-0.024351,-0.016992


In [146]:
df1=ts_washi
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)
df1

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1007.069375,11.374318,-0.000024,6.228312e-06,2.071853e-05,0.000008,-9.523897e-06,-2.231343e-07,314.772888,350.313934,...,-0.008641,-0.022980,-0.019074,-0.026559,-0.027983,-0.029231,-0.018274,-0.007241,0.000964,0.005387
1,1008.814375,12.570400,-0.000014,2.781810e-06,1.150798e-05,0.000003,-8.442781e-06,6.725560e-06,313.111267,350.093811,...,0.003057,-0.015082,-0.013882,-0.015330,-0.021173,-0.027360,-0.008871,0.003124,0.012757,0.017991
2,1009.147500,12.993572,-0.000005,-2.801417e-07,-2.842421e-06,-0.000002,-3.891363e-06,8.958064e-06,313.254547,350.107147,...,0.006204,-0.012347,-0.032648,-0.045578,-0.049795,-0.049760,-0.033698,-0.027660,-0.020196,-0.015848
3,1007.325625,12.251040,-0.000002,1.103165e-05,-8.869802e-08,0.000004,1.414512e-06,6.908057e-06,317.138824,349.477478,...,0.018165,0.000719,-0.023445,-0.042156,-0.051826,-0.055572,-0.049214,-0.041706,-0.030513,-0.024079
4,1006.491875,12.024080,-0.000011,9.125998e-06,3.205580e-06,0.000008,7.640495e-07,6.252454e-06,317.027039,349.305023,...,0.023761,0.008616,-0.004516,-0.013878,-0.015859,-0.014243,-0.010377,-0.004005,0.004443,0.009003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,1004.050000,18.803926,0.000039,1.170732e-05,-2.679585e-07,-0.000002,-7.172116e-07,-4.642721e-06,335.051941,353.788757,...,-0.053408,-0.039242,-0.028068,-0.032141,-0.050734,-0.064892,-0.056993,-0.041778,-0.025674,-0.016432
57,1003.718750,18.780231,0.000031,-3.424115e-06,-1.704585e-05,-0.000010,7.581883e-06,-1.079770e-06,334.113342,353.285675,...,-0.052267,-0.036092,-0.027530,-0.039496,-0.054264,-0.052381,-0.032866,-0.013568,0.003633,0.013292
58,1006.248125,18.004267,0.000015,-5.888951e-06,-1.915601e-05,-0.000010,7.138413e-06,6.689090e-07,331.277893,352.275787,...,-0.046109,-0.043566,-0.049935,-0.058294,-0.056933,-0.039258,-0.023833,-0.007506,0.008584,0.017972
59,1006.571250,16.774679,0.000011,-1.164030e-05,-1.414149e-05,-0.000006,2.098375e-06,-2.144409e-06,334.114502,352.113495,...,-0.057529,-0.053221,-0.055181,-0.069872,-0.074319,-0.065659,-0.053174,-0.038931,-0.024351,-0.016992


In [147]:
df1.to_csv(output+'timeseries_all_wp_washi.csv')

In [95]:
maindf2=pd.read_csv(output+'inner/inner_wp_all_bopha.csv',delimiter=r",")
maindf2.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf2=maindf2.drop('a', axis=1)

pdf_bopha=pd.read_csv(target+'2012_pmin_nwp_bopha.csv',delimiter=r",")
pdf_bopha.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_bopha=pdf_bopha.drop('a', axis=1)

extra=pd.read_csv(output+'nwp/outder_2012_nwp_bopha.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_bopha=pd.DataFrame.from_dict(TCr_ts[4])
prlev_bopha=pd.DataFrame.from_dict(TCpr_ts[4])
slev_bopha=pd.DataFrame.from_dict(TCs_ts[4])

prlev_bopha['outmslp']=prlev_bopha['outmslp']/100

ts1_bopha=pd.concat([maindf2,extra,slev_bopha,rlev_bopha,prlev_bopha], axis=1, join='inner')


tempvortdict = {'outvort_'+plev[i]:TCp_ts[4]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[4]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[4]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[4]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}
plev_bopha=pd.DataFrame.from_dict(alldict)
ts_bopha=pd.concat([pdf_bopha,ts1_bopha,plev_bopha], axis=1, join='inner')
ts_bopha

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1008.131875,11.092641,0.000064,0.000042,0.000023,3.910915e-06,-4.563233e-06,-1.868183e-05,336.073883,350.519592,...,-0.131051,-0.149757,-0.127510,-0.103872,-0.070594,-0.033144,-0.019614,-0.009976,-0.004854,-0.001872
1,1006.676875,9.667654,0.000048,0.000041,0.000025,1.155771e-05,-3.857489e-06,-2.412654e-05,336.479614,349.877228,...,-0.130703,-0.139564,-0.125339,-0.105438,-0.080029,-0.054997,-0.045516,-0.034507,-0.028945,-0.025580
2,1005.663750,9.848610,0.000027,0.000032,0.000026,2.015611e-05,-4.935975e-06,-2.737122e-05,336.045593,349.157166,...,-0.124696,-0.127940,-0.108006,-0.082757,-0.060943,-0.040020,-0.020606,-0.007411,0.001075,0.005375
3,1007.685625,9.333220,0.000014,0.000021,0.000030,2.707488e-05,-2.749592e-06,-1.301638e-05,335.500244,349.245117,...,-0.123161,-0.117384,-0.088698,-0.063757,-0.044755,-0.026221,-0.004939,0.002510,0.010562,0.015513
4,1007.280625,9.189720,0.000013,0.000017,0.000026,1.576553e-05,4.539037e-06,-1.087956e-05,335.900574,350.834473,...,-0.116840,-0.117756,-0.090798,-0.052791,-0.036337,-0.036532,-0.035435,-0.031211,-0.024277,-0.019930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,1003.339375,14.970678,0.000015,-0.000006,-0.000001,8.178898e-07,-1.223705e-05,1.842626e-06,331.840210,348.687927,...,0.008669,0.020327,0.015177,0.001010,-0.032683,-0.042149,-0.034552,-0.026525,-0.017020,-0.011173
107,1003.501875,14.750920,0.000010,-0.000006,-0.000005,7.828398e-06,-6.495432e-06,-1.132198e-06,332.813812,348.504089,...,0.013684,0.023774,0.018477,-0.001256,-0.030733,-0.024747,-0.013742,-0.005221,0.004364,0.009901
108,1006.381875,15.959667,0.000010,-0.000003,-0.000004,6.613144e-06,-1.746871e-06,-8.347175e-07,331.304413,348.503937,...,0.003339,0.015805,0.010192,0.003515,-0.029982,-0.034299,-0.017930,-0.002718,0.006758,0.011055
109,1006.684375,15.455827,0.000004,-0.000003,-0.000003,1.294745e-07,-5.299962e-07,6.249441e-06,332.513489,348.450134,...,0.003544,0.014030,0.001518,-0.007924,-0.035287,-0.049138,-0.056485,-0.037541,-0.027601,-0.025817


In [96]:
df1=ts_bopha
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)
df1

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1008.131875,11.092641,0.000064,0.000042,0.000023,3.910915e-06,-4.563233e-06,-1.868183e-05,336.073883,350.519592,...,-0.131051,-0.149757,-0.127510,-0.103872,-0.070594,-0.033144,-0.019614,-0.009976,-0.004854,-0.001872
1,1006.676875,9.667654,0.000048,0.000041,0.000025,1.155771e-05,-3.857489e-06,-2.412654e-05,336.479614,349.877228,...,-0.130703,-0.139564,-0.125339,-0.105438,-0.080029,-0.054997,-0.045516,-0.034507,-0.028945,-0.025580
2,1005.663750,9.848610,0.000027,0.000032,0.000026,2.015611e-05,-4.935975e-06,-2.737122e-05,336.045593,349.157166,...,-0.124696,-0.127940,-0.108006,-0.082757,-0.060943,-0.040020,-0.020606,-0.007411,0.001075,0.005375
3,1007.685625,9.333220,0.000014,0.000021,0.000030,2.707488e-05,-2.749592e-06,-1.301638e-05,335.500244,349.245117,...,-0.123161,-0.117384,-0.088698,-0.063757,-0.044755,-0.026221,-0.004939,0.002510,0.010562,0.015513
4,1007.280625,9.189720,0.000013,0.000017,0.000026,1.576553e-05,4.539037e-06,-1.087956e-05,335.900574,350.834473,...,-0.116840,-0.117756,-0.090798,-0.052791,-0.036337,-0.036532,-0.035435,-0.031211,-0.024277,-0.019930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,1003.339375,14.970678,0.000015,-0.000006,-0.000001,8.178898e-07,-1.223705e-05,1.842626e-06,331.840210,348.687927,...,0.008669,0.020327,0.015177,0.001010,-0.032683,-0.042149,-0.034552,-0.026525,-0.017020,-0.011173
107,1003.501875,14.750920,0.000010,-0.000006,-0.000005,7.828398e-06,-6.495432e-06,-1.132198e-06,332.813812,348.504089,...,0.013684,0.023774,0.018477,-0.001256,-0.030733,-0.024747,-0.013742,-0.005221,0.004364,0.009901
108,1006.381875,15.959667,0.000010,-0.000003,-0.000004,6.613144e-06,-1.746871e-06,-8.347175e-07,331.304413,348.503937,...,0.003339,0.015805,0.010192,0.003515,-0.029982,-0.034299,-0.017930,-0.002718,0.006758,0.011055
109,1006.684375,15.455827,0.000004,-0.000003,-0.000003,1.294745e-07,-5.299962e-07,6.249441e-06,332.513489,348.450134,...,0.003544,0.014030,0.001518,-0.007924,-0.035287,-0.049138,-0.056485,-0.037541,-0.027601,-0.025817


In [97]:
df1.to_csv(output+'timeseries_all_wp_bopha.csv')

In [64]:
maindf2=pd.read_csv(output+'inner/inner_wp_all_haiyan.csv',delimiter=r",")
maindf2.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf2=maindf2.drop('a', axis=1)

pdf_haiyan=pd.read_csv(target+'2013_pmin_nwp_haiyan.csv',delimiter=r",")
pdf_haiyan.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_haiyan=pdf_haiyan.drop('a', axis=1)

extra=pd.read_csv(output+'nwp/outder_2013_nwp_haiyan.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_haiyan=pd.DataFrame.from_dict(TCr_ts[5])
prlev_haiyan=pd.DataFrame.from_dict(TCpr_ts[5])
slev_haiyan=pd.DataFrame.from_dict(TCs_ts[5])

prlev_haiyan['outmslp']=prlev_haiyan['outmslp']/100

ts1_haiyan=pd.concat([maindf2,extra,slev_haiyan,rlev_haiyan,prlev_haiyan], axis=1, join='inner')


tempvortdict = {'outvort_'+plev[i]:TCp_ts[5]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[5]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[5]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[5]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}
plev_haiyan=pd.DataFrame.from_dict(alldict)
ts_haiyan=pd.concat([pdf_haiyan,ts1_haiyan,plev_haiyan], axis=1, join='inner')
ts_haiyan

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1009.146875,8.531227,0.000001,0.000023,1.902287e-05,0.000006,-0.000020,0.000003,328.512970,348.270355,...,-0.030995,-0.029351,-0.021178,-0.014323,-0.018814,-0.018528,-0.013186,-0.003382,0.007463,0.012723
1,1007.724375,8.789506,-0.000005,0.000022,1.894513e-05,0.000012,-0.000015,0.000003,326.717407,347.581848,...,-0.033258,-0.030404,-0.016700,-0.014649,-0.021834,-0.026210,-0.017385,-0.004354,0.008519,0.014372
2,1007.572500,9.109775,0.000009,0.000026,2.287460e-05,0.000014,-0.000004,0.000008,332.980713,348.626038,...,-0.038864,-0.035484,-0.025537,-0.035246,-0.049828,-0.057302,-0.046657,-0.031051,-0.016574,-0.009144
3,1008.985000,8.735093,0.000013,0.000024,2.707223e-05,0.000014,-0.000006,0.000002,329.524536,349.692078,...,-0.012157,-0.008206,-0.004862,-0.023503,-0.056912,-0.074630,-0.063489,-0.043353,-0.026451,-0.017984
4,1008.954375,8.762020,0.000021,0.000021,1.660212e-05,0.000006,-0.000002,0.000004,329.931396,349.661682,...,-0.031849,-0.039365,-0.056697,-0.068964,-0.092253,-0.101182,-0.068980,-0.035407,-0.009946,0.002440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,1001.594375,18.393751,0.000027,0.000055,4.400695e-05,0.000029,0.000012,0.000009,324.850250,353.000488,...,-0.048573,-0.069571,-0.096487,-0.078481,-0.083979,-0.073802,-0.008468,0.011669,0.029770,0.037467
71,1004.656250,16.716141,-0.000003,0.000045,2.842371e-05,0.000028,0.000045,0.000027,319.582214,353.327454,...,-0.066405,-0.097735,-0.110775,-0.068281,-0.079841,-0.085351,-0.050199,-0.039796,-0.029832,-0.024306
72,1007.023125,13.369188,-0.000004,0.000025,1.420011e-05,0.000014,0.000049,0.000032,315.080811,353.871429,...,-0.082643,-0.115647,-0.123922,-0.075876,-0.081456,-0.083748,-0.056181,-0.048405,-0.040780,-0.036915
73,1008.596875,11.980754,0.000005,0.000010,9.524666e-06,0.000014,0.000040,0.000030,314.847198,353.513214,...,-0.056677,-0.084635,-0.093489,-0.058088,-0.063104,-0.065794,-0.044773,-0.030567,-0.017712,-0.011977


In [65]:
df1=ts_haiyan
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)
df1

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1009.146875,8.531227,0.000001,0.000023,1.902287e-05,0.000006,-0.000020,0.000003,328.512970,348.270355,...,-0.030995,-0.029351,-0.021178,-0.014323,-0.018814,-0.018528,-0.013186,-0.003382,0.007463,0.012723
1,1007.724375,8.789506,-0.000005,0.000022,1.894513e-05,0.000012,-0.000015,0.000003,326.717407,347.581848,...,-0.033258,-0.030404,-0.016700,-0.014649,-0.021834,-0.026210,-0.017385,-0.004354,0.008519,0.014372
2,1007.572500,9.109775,0.000009,0.000026,2.287460e-05,0.000014,-0.000004,0.000008,332.980713,348.626038,...,-0.038864,-0.035484,-0.025537,-0.035246,-0.049828,-0.057302,-0.046657,-0.031051,-0.016574,-0.009144
3,1008.985000,8.735093,0.000013,0.000024,2.707223e-05,0.000014,-0.000006,0.000002,329.524536,349.692078,...,-0.012157,-0.008206,-0.004862,-0.023503,-0.056912,-0.074630,-0.063489,-0.043353,-0.026451,-0.017984
4,1008.954375,8.762020,0.000021,0.000021,1.660212e-05,0.000006,-0.000002,0.000004,329.931396,349.661682,...,-0.031849,-0.039365,-0.056697,-0.068964,-0.092253,-0.101182,-0.068980,-0.035407,-0.009946,0.002440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,1001.594375,18.393751,0.000027,0.000055,4.400695e-05,0.000029,0.000012,0.000009,324.850250,353.000488,...,-0.048573,-0.069571,-0.096487,-0.078481,-0.083979,-0.073802,-0.008468,0.011669,0.029770,0.037467
71,1004.656250,16.716141,-0.000003,0.000045,2.842371e-05,0.000028,0.000045,0.000027,319.582214,353.327454,...,-0.066405,-0.097735,-0.110775,-0.068281,-0.079841,-0.085351,-0.050199,-0.039796,-0.029832,-0.024306
72,1007.023125,13.369188,-0.000004,0.000025,1.420011e-05,0.000014,0.000049,0.000032,315.080811,353.871429,...,-0.082643,-0.115647,-0.123922,-0.075876,-0.081456,-0.083748,-0.056181,-0.048405,-0.040780,-0.036915
73,1008.596875,11.980754,0.000005,0.000010,9.524666e-06,0.000014,0.000040,0.000030,314.847198,353.513214,...,-0.056677,-0.084635,-0.093489,-0.058088,-0.063104,-0.065794,-0.044773,-0.030567,-0.017712,-0.011977


In [66]:
df1.to_csv(output+'timeseries_all_wp_haiyan.csv')

In [110]:
x=

NameError: name 'ts_koppu' is not defined